In [ ]:
######### for this program to give accurate results, combined_df has to be run again if any code/name changes are there or
########## including any additions
########## companymaster has to be uptodate
######### shortlisted stocks - averages for nse/bse has to be updated
######## market cap from screener has to be updated

In [1]:
# importing the required modules

import os
import csv
import pandas as pd
from pathlib import Path
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, Paragraph, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, TA_LEFT

from platform import python_version
python_version()

'3.9.16'

In [2]:
### this program gives top 50 deliveries by value and volume for fno and non-fno. 
#### it also creates fno_csv and non_fno_csv which is used by multiple scrips program

### caution - if data file is already read, then it skips reading it. So if data is updated, close the program and restart it
#### to enable data file to be read again


In [3]:
base_folder = Path.cwd()
base_folder

WindowsPath('c:/Users/birap/OneDrive/Desktop/Tradeview/tradingView/Python code for chatGpt')

In [4]:
# list of file paths

master_file_path = Path.joinpath(base_folder, "Master File", "CompanyMaster.csv")
directory = Path.joinpath(Path(base_folder).parents[0], "Data Files")
output_path = Path.joinpath(base_folder, "Output File")
combined_file_path = Path.joinpath(Path(base_folder).parents[0], "Delivery Price Combined Files")

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']

In [65]:
file_names = ['BSE Gross Deliverable.txt', 'NSE Delivery.csv', 'NSE EQ.csv', 'EQ ISIN.csv']

while True:
    user_input = input("Enter a date in the 'yymmdd' format (or press Enter to exit): ")

    # Check if the user pressed Enter with an empty input
    if not user_input:
        print("Exiting the program.")
        break

    # Check if the input has 6 characters and consists of digits
    if len(user_input) == 6 and user_input.isdigit():
        all_files_exist = True
        updated_file_names = []

        for file_name in file_names:
            name, extension = file_name.split(".")
            print(name, extension)
            full_file_name = os.path.join(directory, f"{name} {user_input}.{extension}")
            file_name_date = f"{name} {user_input}.{extension}"
            print(f"full file name : {full_file_name} and file_name_date : {file_name_date}")
            if not os.path.exists(full_file_name):
                print()
                all_files_exist = False
                break
            updated_file_names.append(file_name_date)

        if all_files_exist:
            print(f"All required files for the date '{user_input}' exist.")
            # You can use the updated_file_names list for further processing
            print("Updated file names:", updated_file_names)
            break  # Exit the loop if all files exist
        else:
            print("Not all required files exist. Please check the files.")
    else:
        print("Invalid input. Please enter a date in the 'yymmdd' format (e.g., 220101).")
        

BSE Gross Deliverable txt
full file name : c:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Data Files\BSE Gross Deliverable 230810.txt and file_name_date : BSE Gross Deliverable 230810.txt
NSE Delivery csv
full file name : c:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Data Files\NSE Delivery 230810.csv and file_name_date : NSE Delivery 230810.csv
NSE EQ csv
full file name : c:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Data Files\NSE EQ 230810.csv and file_name_date : NSE EQ 230810.csv
EQ ISIN csv
full file name : c:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Data Files\EQ ISIN 230810.csv and file_name_date : EQ ISIN 230810.csv
All required files for the date '230810' exist.
Updated file names: ['BSE Gross Deliverable 230810.txt', 'NSE Delivery 230810.csv', 'NSE EQ 230810.csv', 'EQ ISIN 230810.csv']


In [66]:
nse_dely_file = [updated_file_names[1]]
nse_price_file = [updated_file_names[2]]
bse_dely_file = [updated_file_names[0]]
bse_price_file = [updated_file_names[3]]
print(nse_dely_file,nse_price_file,bse_dely_file,bse_price_file)

['NSE Delivery 230810.csv'] ['NSE EQ 230810.csv'] ['BSE Gross Deliverable 230810.txt'] ['EQ ISIN 230810.csv']


In [67]:
def nse_data(nse_file_name):
    nse_df = pd.DataFrame(columns=['NSE_Code','Date','NSE_Volume','NSE_Delivery','Segment'])
    dely_segment = ['EQ','BE','SM','ST']
    
    for file_name in nse_file_name:
        file_path = os.path.join(directory, file_name)
        date = file_name[13:19]
        
        with open(file_path,'r') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader, start=1):
                if i >= 5:
                    line_str = ','.join(line)
                    data = line_str.split(',')
                    if data[3] in dely_segment:
                        nse_code = data[2]
                        dely = data[5]
                        segment = data[3]
                        volume = data[4]
                        df_to_append = pd.DataFrame({'NSE_Code': [nse_code], 'Date': [date], 'NSE_Volume':volume,
                                                     'NSE_Delivery': [dely], 'Segment': [segment]})
                        nse_df = pd.concat([nse_df, df_to_append], ignore_index=True)
    return nse_df
                
 
def bse_data(bse_file_name):
    # Create an empty DataFrame to store the results
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date','BSE_Delivery'])
    dely_segment = ['EQ','BE','SM','ST']

    for file_name in bse_file_name:
        file_path = os.path.join(directory, file_name)
        date = file_name[22:28] 
    
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader, start=1):
                if i >= 2:  # Start reading from line 5
                    line_str = ','.join(line)
                    data = line_str.split('|')
                    dely = data[2]
                    bse_code = data[1]
                    df_to_append = pd.DataFrame({'BSE_Code': [bse_code], 'Date': [date], 'BSE_Delivery':[dely]}) 
                    bse_df = pd.concat([bse_df,df_to_append], ignore_index=True)               
    return bse_df

In [68]:
def nse_price(nse_price_file):
    nse_df = pd.DataFrame(columns=['NSE_Code', 'Date', 'Price', 'PrevClose', 'Segment', 'Traded'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']

    for file_name in nse_price_file:
        file_path = os.path.join(directory, file_name)
        date = file_name[7:13]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row and row[1] in dely_segment:  # Check if the row is not empty and segment matches
                    nse_code = row[0]
                    price = row[5]
                    prev_close = row[7]
                    segment = row[1]
                    traded = row[8]
                    df_to_append = pd.DataFrame({'NSE_Code': [nse_code], 'Date': [date], 'Price': [price], 
                                                 'PrevClose': [prev_close],'Segment': [segment], 'Traded': [traded]})
                    nse_df = pd.concat([nse_df, df_to_append], ignore_index=True)
    return nse_df

def bse_price(bse_price_file):
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date', 'Price', 'PrevClose', 'BSE_Volume','Segment'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']

    for file_name in bse_price_file:
        file_path = os.path.join(directory, file_name)
        date = file_name[8:14]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                #if row and row[1] in dely_segment:  # Check if the row is not empty and segment matches
                    bse_code = row[0]
                    price = row[7]
                    prev_close = row[9]
                    bse_group = row[2]
                    volume = row[11]
                    df_to_append = pd.DataFrame(
                                {'BSE_Code': [bse_code], 
                                'Date': [date], 
                                'Price': [price], 
                                'PrevClose': [prev_close],
                                'BSE_Volume':volume ,
                                'Segment': [bse_group]}
                             )
                    bse_df = pd.concat([bse_df, df_to_append], ignore_index=True)

    # Drop the first row, which is a header row, and reset the index
    bse_df = bse_df.iloc[1:].reset_index(drop=True)
    return bse_df

In [69]:
def add_nse_prefix(df, column_name):
    """Create a copy of the DataFrame to avoid SettingWithCopyWarning"""
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)
    return df

In [70]:
### getting unique symbols from fno
nse_fno_file = sorted([file for file in os.listdir(directory) if file.startswith('NSE FNO')],reverse=True)[:1]

for file_name in nse_fno_file:
    file_path = os.path.join(directory, file_name)
    fno_df = pd.read_csv(file_path)
    fno_df = fno_df.drop_duplicates(subset ='SYMBOL')
    fno_symbols = fno_df['SYMBOL']
    fno_symbols = fno_symbols.reset_index(drop=True)

In [71]:
print('processing nse_dely')
nse_dely_data = nse_data(nse_dely_file)

print('processing bse_dely')
bse_dely_data = bse_data(bse_dely_file)

print('processing nse_price')
nse_price_data = nse_price(nse_price_file)

print('processing bse_price')
bse_price_data = bse_price(bse_price_file)

print('all done, proceed')

processing nse_dely
processing bse_dely
processing nse_price
processing bse_price
all done, proceed


In [72]:
nse_dely_data['NSE_Delivery'] = nse_dely_data['NSE_Delivery'].astype(int)
nse_dely_data['NSE_Volume'] = nse_dely_data['NSE_Volume'].astype(int)
nse_price_data['Price'] = nse_price_data['Price'].astype(float)
nse_price_data['PrevClose'] = nse_price_data['PrevClose'].astype(float)
nse_price_data['Traded'] = nse_price_data['Traded'].astype(int)

bse_dely_data['BSE_Delivery'] = bse_dely_data['BSE_Delivery'].astype(int)
bse_price_data['Price'] = bse_price_data['Price'].astype(float)
bse_price_data['PrevClose'] = bse_price_data['PrevClose'].astype(float)
bse_price_data['BSE_Volume'] = bse_price_data['BSE_Volume'].astype(int)


In [73]:
# nse_dely_data.info()
# bse_dely_data.info()
# nse_price_data.info()
# bse_price_data.info()

In [74]:
### nse_dely_data has NSE_Code, Date, NSE_Delivery, Segment
### bse_dely_data has BSE_Code, Date, BSE_Delivery

### nse_price_data has NSE_Code	Date	Price	PrevClose	Segment	Traded
### bse_price_data has BSE_Code	Date	Price	PrevClose	Segment

In [75]:
## this section processes nse delivery and price file. Makes sure that stocks which are in BE segment and 
## therefore dont appear in nse delivery are taken from price file and new nse merged df is generated

### merging nse price and delivery files
nse_merged_df_out = pd.merge(nse_dely_data,nse_price_data, on = 'NSE_Code', how = 'outer')

### processing merged file
# Replace 'NSE_Delivery' with 'Traded' where 'NSE_Delivery' is NaN
nse_merged_df_out['NSE_Delivery'] = nse_merged_df_out['NSE_Delivery'].fillna(nse_merged_df_out['Traded'])

# Replace nse volume with nse delivery where nse volume is NaN
nse_merged_df_out['NSE_Volume'] = nse_merged_df_out['NSE_Volume'].fillna(nse_merged_df_out['NSE_Delivery'])

# Replace 'Date_x' with 'Date_y' where 'Date_x' is NaN
nse_merged_df_out['Date_x'] = nse_merged_df_out['Date_x'].fillna(nse_merged_df_out['Date_y'])

# Replace 'Segment_x' with 'Segment_y' where 'Segment_x' is NaN
nse_merged_df_out['Segment_x'] = nse_merged_df_out['Segment_x'].fillna(nse_merged_df_out['Segment_y'])

# Rename 'Segment_x' to 'Segment'
nse_merged_df_out.rename(columns={'Segment_x': 'Segment'}, inplace=True)

# Rename 'Date_x' to 'Date'
nse_merged_df_out.rename(columns={'Date_x': 'Date'}, inplace=True)

# Drop the 'Date_y' column
nse_merged_df_out.drop(columns=['Date_y'], inplace=True)

# Drop the 'Segment_y' column
nse_merged_df_out.drop(columns=['Segment_y'], inplace=True)

# Delete the 'Traded' column after replacement
nse_merged_df_out.drop(columns=['Traded'], inplace=True)

# Reset the index
nse_merged_df_out.reset_index(drop=True, inplace=True)

# add traded_value column

nse_merged_df_out['Traded_Value_lacs'] = (nse_merged_df_out['NSE_Delivery'] * nse_merged_df_out['Price'] / 100000).round(2)

#print(nse_merged_df_out)

nse_col_order = ['Date','NSE_Code','NSE_Volume','NSE_Delivery','Price','PrevClose','Traded_Value_lacs','Segment']
nse_merged_df_out = nse_merged_df_out[nse_col_order]

#### nse_merged_df_out is NSE_Code	Date	NSE_Delivery	Segment	Price	PrevClose	Traded_Value_lacs


In [76]:
nse_merged_df_out.head()

,Date,NSE_Code,NSE_Volume,NSE_Delivery,Price,PrevClose,Traded_Value_lacs,Segment
0,230810,21STCENMGM,2931.0,2779.0,20.40,20.65,0.57,EQ
1,230810,360ONE,330518.0,252017.0,510.15,506.40,1285.66,EQ
2,230810,3IINFOLTD,699311.0,466129.0,33.00,33.35,153.82,EQ
3,230810,3MINDIA,17315.0,3677.0,28119.45,26828.15,1033.95,EQ
4,230810,5PAISA,37455.0,16972.0,459.35,458.75,77.96,EQ


In [77]:
###merging bse price and delivery files

bse_merged_df_out = pd.merge(bse_dely_data, bse_price_data, on = 'BSE_Code', how = 'outer')


### processing bse merged file

bse_merged_df_out['Traded_Value_lacs'] = (bse_merged_df_out['BSE_Delivery'] * bse_merged_df_out['Price'] / 100000).round(2)

# Rename 'Date_x' to 'Date'
bse_merged_df_out.rename(columns={'Date_x': 'Date'}, inplace=True)

# Drop the 'Date_y' column
bse_merged_df_out.drop(columns=['Date_y'], inplace=True)
bse_merged_df_out = bse_merged_df_out[bse_merged_df_out['BSE_Code'] <= '599999']

bse_merged_df_out.reset_index(drop=True, inplace = True)

#print(bse_merged_df_out)

bse_col_order = ['Date','BSE_Code','BSE_Volume','BSE_Delivery','Price','PrevClose','Traded_Value_lacs','Segment']
bse_merged_df_out = bse_merged_df_out[bse_col_order]

#### bse_merged_df_out is BSE_Code	Date	BSE_Delivery	Price	PrevClose	Segment	Traded_Value_lacs

In [78]:
### saving nse and bse files separately

nse_file = str(combined_file_path) +"\\"+ 'nse_' + user_input + '.csv'
bse_file = str(combined_file_path) + "\\"+'bse_' + user_input + '.csv'

#df.to_csv('sample.csv', index=False)

nse_merged_df_out.to_csv(nse_file, index = False)
bse_merged_df_out.to_csv(bse_file, index = False)


In [79]:
#nse_merged_df_out.info()
#nse_merged_df_out

In [80]:
#bse_merged_df_out.info()
#bse_merged_df_out

In [81]:
## reading master file
### master file (csv) is Company Name	CD_NSE Symbol	CD_BSE Code	CD_Is Subsidiary	CD_Sector	CD_Industry1

#### TO FIX - MASTER FILE DOES NOT HAVE SM SEGMENT CODES

master_file = pd.read_csv(master_file_path, dtype={'CD_BSE Code': str})


###df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
##df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
### Inplace method (modifies the original DataFrame)
###df.drop(columns=['column1', 'column2'], inplace=True)

master_file.drop(columns=['CD_Is Subsidiary','CD_Sector','CD_Industry1'], inplace = True )
master_file.rename(columns={'CD_NSE Symbol': 'NSE_Code', 'CD_BSE Code': 'BSE_Code'}, inplace= True )


#### master_file df is Company Name	NSE_Code	BSE_Code

#print(master_file)

In [82]:
## Merge NSE df with master file 

master_file = pd.merge(master_file,nse_merged_df_out, on= "NSE_Code", how = 'outer')
#master_file.dropna(subset=['Company Name'], inplace=True)


## merge bse df  with master file 
master_file = pd.merge(master_file,bse_merged_df_out, on ="BSE_Code", how = "outer")
#master_file.dropna(subset=['Company Name'], inplace=True)

# Drop records where both 'NSE_Code' and 'BSE_Code' are NaN or blank
master_file.dropna(subset=['NSE_Code', 'BSE_Code'], how='all', inplace=True)

# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)

# Drop records where both 'NSE_Delivery' and 'BSE_Delivery' are 0 or blank or NaN
master_file = master_file[~((master_file['NSE_Delivery'] == 0) & (master_file['BSE_Delivery'] == 0))]

# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)


# Trim "Company Name" column to a maximum length of 20 characters
master_file['Company Name'] = master_file['Company Name'].str.slice(0, 30)


# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)

#print(master_file)

In [83]:

# Identify the data types of each column
column_data_types = master_file.dtypes

# Replace NaN values in object columns with empty strings ("")
object_columns = master_file.select_dtypes(include='object').columns
master_file[object_columns] = master_file[object_columns].fillna("")

# Replace NaN values in float/integer columns with 0
numeric_columns = master_file.select_dtypes(include=['float64', 'int64']).columns
master_file[numeric_columns] = master_file[numeric_columns].fillna(0)

# Print the resulting DataFrame
#print(master_file)

#master_file.info()

In [84]:
# Define a custom function to replace Price_x with Price_y if Price_x is 0
def replace_price_x(row):
    if row['Price_x'] == 0:
        return row['Price_y']
    else:
        return row['Price_x']

    
def replace_prevclose_x(row):
    if row['PrevClose_x'] == 0:
        return row['PrevClose_y']
    else:
        return row['PrevClose_x']
    
    
def replace_segment(row):
    if row['Segment_x'] == "":
        return row['Segment_y']
    else:
        return row['Segment_x']
    
def replace_date(row):
    if row['Date_x'] == "":
        return row['Date_y']
    else:
        return row['Date_x']

# Apply the custom function to the DataFrame
master_file['Price_x'] = master_file.apply(replace_price_x, axis=1)
master_file['PrevClose_x'] = master_file.apply(replace_prevclose_x, axis=1)

# Rename 'Price_x' column to 'Price'
master_file.rename(columns={'Price_x': 'Price'}, inplace=True)
master_file.rename(columns={'PrevClose_x':'PrevClose'}, inplace=True)

### segment

master_file['Segment_x'] = master_file.apply(replace_segment, axis = 1)
master_file.rename(columns={'Segment_x' :'Segment'}, inplace = True)

#Date
master_file['Date_x'] = master_file.apply(replace_date, axis = 1)
master_file.rename(columns={'Date_x' :'Date'}, inplace = True)

### add total delivery column
master_file['Total_Delivery'] = master_file['NSE_Delivery'] + master_file['BSE_Delivery']
master_file['Total_Delivery'] = master_file['Total_Delivery'].astype(int)
master_file = master_file[master_file['Total_Delivery'] >= 1]

### add total volume column
master_file['Total_Volume'] = master_file['NSE_Volume'] + master_file['BSE_Volume']
master_file['Total_Volume'] = master_file['Total_Volume'].astype(int)


## add total traded value colum

master_file['Total_Traded_Value'] = ( master_file['Total_Delivery'] * master_file['Price'] ) / 100000



### reorder columns

columns_to_copy = ['Date','Company Name','Total_Volume','Total_Delivery', 'Price','PrevClose','Total_Traded_Value' ,'NSE_Code','BSE_Code','Segment']

# Create a new DataFrame 'final_master_file' with the desired columns
#final_master_file = master_file[columns_to_copy].copy()



In [85]:
#master_file

In [86]:

master_file_columns = ['Date','Company Name','NSE_Code','BSE_Code','Total_Volume','Total_Delivery', 'Price','PrevClose',
                       'Total_Traded_Value' , 'NSE_Volume','NSE_Delivery','BSE_Volume','BSE_Delivery','Segment']
final_master_file = master_file[master_file_columns]
master_file_csv = str(combined_file_path) + "\\"+'combined_' + user_input + '.csv'

#df.to_csv('sample.csv', index=False)

final_master_file.to_csv(master_file_csv, index = False)

In [87]:
#### create sme file
sme_segment = ['SM','ST','M ','MT']
sme_delivery = final_master_file[final_master_file['Segment'].isin(sme_segment)]
sme_delivery.loc[sme_delivery['NSE_Code'] == '', 'NSE_Code'] = sme_delivery['BSE_Code']
sme_delivery.reset_index(drop=True, inplace=True)

# Drop records where Total_Traded_Value is below 1000
final_master_file = final_master_file[final_master_file['Total_Traded_Value'] >= 1000]


In [88]:
###creating separate file for fno delivery and non fno delivery

# Filter final_master_file to get fno_delivery by checking if NSE_Code is in fno_symbols
fno_delivery = final_master_file[final_master_file['NSE_Code'].isin(fno_symbols)]
fno_delivery_final = fno_delivery[['NSE_Code', 'NSE_Volume','Total_Delivery', 'Price','PrevClose', 
                                   'Total_Traded_Value']].copy()


# Create non_fno_delivery by dropping rows where NSE_Code is in fno_symbols
non_fno_delivery = final_master_file[~final_master_file['NSE_Code'].isin(fno_symbols)]

# Reset index inplace for fno_delivery and non_fno_delivery
fno_delivery_final.reset_index(drop=True, inplace=True)
non_fno_delivery.reset_index(drop=True, inplace=True)

# Replace empty strings in NSE_Code with BSE_Code
#non_fno_delivery.loc[non_fno_delivery['NSE_Code'] == '', 'NSE_Code'] = non_fno_delivery['BSE_Code']


# Replace empty strings in NSE_Code with BSE_Code using a copy
non_fno_delivery_copy = non_fno_delivery.copy()
non_fno_delivery_copy.loc[non_fno_delivery_copy['NSE_Code'] == '', 'NSE_Code'] = non_fno_delivery_copy['BSE_Code']

In [89]:
# Replace empty strings in NSE_Code with BSE_Code
#non_fno_delivery['NSE_Code'] = non_fno_delivery.apply(lambda row: row['BSE_Code'] if row['NSE_Code'] == "" else row['NSE_Code'], axis=1)

#non_fno_delivery.loc[non_fno_delivery['NSE_Code'] == '', 'NSE_Code'] = non_fno_delivery['BSE_Code']


#sme_delivery
#non_fno_delivery

In [90]:
segment_var = ['SM','ST','M ']
#final_master_file.info()
 ##Count the records in each segment
segment_counts = master_file['Segment'].value_counts()

#print(segment_counts)

In [91]:
nse_dely_date = user_input
### creating top 50 by value and by volume for fno and non-fno

def top_n(df, n, column_name, trade_amt):
    df_filtered = df[df['Total_Traded_Value'] > trade_amt]
    top_n = df_filtered.nlargest(n, column_name).sort_values(column_name, ascending=False)
    
    # Format and right-align specific columns
    columns_to_format = ['Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value']
    column_widths = {'Total_Delivery': 15, 'Price': 12, 'PrevClose': 12, 'Total_Traded_Value': 18}

    for col in columns_to_format:
        if col == 'Total_Delivery':
            top_n[col] = top_n[col].apply(lambda x: f"{int(x):,}".rjust(column_widths[col]))
        else:
            top_n[col] = top_n[col].apply(lambda x: f"{x:,.2f}".rjust(column_widths[col]))

    return top_n



df_value_50 = top_n(non_fno_delivery,50,'Total_Traded_Value', 250)
df_volume_50 = top_n(non_fno_delivery,50,'Total_Delivery', 250)

fno_value_50 = top_n(fno_delivery,50,'Total_Traded_Value', 1000)
fno_volume_50 = top_n(fno_delivery,50,'Total_Delivery', 1000)

fno_columns = ['NSE_Code', 'Total_Delivery','Price','PrevClose','Total_Traded_Value']

fno_value_50 = fno_value_50[fno_columns]
fno_volume_50 = fno_volume_50[fno_columns]

sme_value_50 = top_n(sme_delivery,50,'Total_Traded_Value', 50)

###########################

###########################

## write to excel -- non_fno value and volume is merged by concat and saved for multiple scrips report (7-day history)
#### sme_csv is similarly saved for 7-day history report

df_non_fno = pd.concat([df_value_50, df_volume_50])
df_non_fno_date = df_non_fno
df_non_fno_wl = df_non_fno[['NSE_Code']]

df_fno = pd.concat([fno_value_50, fno_volume_50])

# Reset the index of the series
df_non_fno_wl.reset_index(drop=True, inplace=True)

#add NSE: prefix to NSE_Code
df_non_fno_wl_nse = add_nse_prefix(df_non_fno_wl,'NSE_Code')


### define csv files
output_non_fno = str(output_path) +'\\'+"0_non_fno.csv"
output_non_fno_date = str(output_path) + "\\"+nse_dely_date + "_0_non_fno_.csv"
output_non_fno_wl = str(output_path) + "\\"+ '_' + nse_dely_date +"_0_non_fno_wl.csv"


output_fno = str(output_path) + "\\"+ "fno.csv"

output_sme_nodate = str(output_path)+ "\\" + 'sme_csv.csv'
output_sme = str(output_path)+ "\\" + nse_dely_date + '_0_sme_.csv'

### write to csv
df_non_fno.to_csv(output_non_fno, index = False)
df_non_fno.to_csv(output_non_fno_date, index = False)
df_non_fno_wl_nse.to_csv(output_non_fno_wl, index = False)

df_fno.to_csv(output_fno, index = False)

sme_delivery.to_csv(output_sme, index=False)
sme_value_50.to_csv(output_sme_nodate)

In [92]:
df_non_fno.head()

,Date,Company Name,NSE_Code,BSE_Code,Total_Volume,Total_Delivery,Price,PrevClose,Total_Traded_Value,NSE_Volume,NSE_Delivery,BSE_Volume,BSE_Delivery,Segment
63,230810,CMS Info Systems Ltd.,CMSINFO,543441,49598157,"37,746,719",362.75,390.80,"136,926.22",16060039.0,7688884.0,33538118.0,30057835.0,EQ
312,230810,Zomato Ltd.,ZOMATO,543320,92180496,"32,149,240",94.25,94.10,"30,300.66",87598556.0,30631170.0,4581940.0,1518070.0,EQ
4,230810,,ADANITRANS,,6591528,"3,394,587",820.80,811.95,"27,862.77",6591528.0,3394587.0,0.0,0.0,EQ
266,230810,Suzlon Energy Ltd.,SUZLON,532667,252673762,"125,213,901",20.45,19.50,"25,606.24",198521194.0,93010628.0,54152568.0,32203273.0,EQ
161,230810,K.P.R. Mill Ltd.,KPRMILL,532889,4294871,"3,822,055",661.45,633.20,"25,280.98",2462328.0,2010189.0,1832543.0,1811866.0,EQ


In [93]:
def print_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = row[columns_to_select[0]].ljust(40)
        p_dely = f"{row[columns_to_select[1]]:,.0f}".rjust(15)
        p_price = f"{row[columns_to_select[2]]:,.2f}".rjust(12)
        p_prevclose = f"{row[columns_to_select[3]]:,.2f}".rjust(12)
        p_trade_value = f"{row[columns_to_select[4]]:,.2f}".rjust(18)
        p_nse_code = row[columns_to_select[5]].ljust(10)
        
        print(f"{p_coname}{p_dely}{p_price}{p_prevclose}{p_trade_value}    {p_nse_code}")

        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

def fno_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(),1):
        p_nse_code = row[columns_to_select[0]].ljust(10)
        p_dely = f"{row[columns_to_select[1]]:,.0f}".rjust(15)
        p_price = f"{row[columns_to_select[2]]:,.2f}".rjust(12)
        p_prevclose = f"{row[columns_to_select[3]]:,.2f}".rjust(12)
        p_trade_value = f"{row[columns_to_select[4]]:,.2f}".rjust(12)
        
        print(f"{p_nse_code}{p_dely}{p_price}{p_prevclose}{p_trade_value}")
        
        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

In [94]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph
from pathlib import Path
from reportlab.pdfbase.pdfmetrics import registerFontFamily

registerFontFamily('Vera', normal='Vera', bold='VeraBd', italic='VeraIt', boldItalic='VeraBI')

columns_to_select = ['Company Name', 'Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value', 'NSE_Code']
fno_columns_to_select = ['NSE_Code', 'Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value']
dfs = [df_value_50, df_volume_50, fno_value_50, fno_volume_50, sme_value_50]
report_titles = [
    f"Top 50 delivery by value NON-FNO {user_input}",
    f"Top 50 delivery by volume NON-FNO {user_input}",
    f"Top 50 FNO Delivery by value {user_input}",
    f"Top 50 FNO Delivery by volume {user_input}",
    f"Top 50 delivery by value {user_input}"
]

report_prefix_name = f"51-script-report_{str(user_input)}.pdf"
report_path = Path.joinpath(Path(base_folder).parents[0], "Reports", report_prefix_name)

# Define the style for column headers without grid lines
column_header_style = [
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
]


def create_table_with_header(df, df_name, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()

    # Add empty rows for spacing after every 5 records
    for i in range(6, len(table_data), 6):
        table_data.insert(i, [''] * len(df.columns))
        
    styles = getSampleStyleSheet()
    h2_style = styles['Normal']
    h2_style.fontSize = 10
    h2_style.fontName = 'Calibri'

    if table_width is None:
        t = Table(table_data, repeatRows=1, style=h2_style)
    else:
        colwdth = [170, 80, 67, 68, 105]
        t = Table(table_data, repeatRows=1, colWidths=colwdth)  # Adjust colWidths for left alignment

    style = [
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left align first column
        ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align last column
        ('ALIGN', (1, 0), (-2, -1), 'RIGHT'),  # Right align other columns
    ]

    # repeatRows=1 to add the first column to all the pages
    t.setStyle(column_header_style)

    if "FNO" in str(df_name) and "NON-FNO" not in str(df_name):
        style = [
            ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align first column
            ('ALIGN', (1, 0), (-1, -1), 'RIGHT'),  # Right align other columns (including last)
        ]

    # Set title row color for the first page
    if getattr(create_table_with_header, 'first_page', True):
        style.extend([
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),  # Background color for the title row
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ])
        create_table_with_header.first_page = False

    t.setStyle(style)

    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white)])  # Hide grid lines

    return t


def add_headers_and_footers(canvas, doc):
    """Add header and footer here"""
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    footer_style = getSampleStyleSheet()['Normal']
    footer_style.alignment = 1  # 0=Left, 1=Center, 2=Right

    footer = Paragraph(text, footer_style)
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, doc.bottomMargin - h)

    # Change title text color to white for pages after the first one
    if page_num > 1:
        title_style = getSampleStyleSheet()['Title']
        title_style.textColor = colors.white
        main_title.style = title_style


left_margin = 20
right_margin = 20
top_margin = 5
bottom_margin = 20  # Add a bottom margin
page_width = letter[0]  # Get width from letter page size
page_height = letter[1] - top_margin - bottom_margin  # Adjust height for margins

# Create a SimpleDocTemplate with adjusted page size and margins
doc = SimpleDocTemplate(
    str(report_path),
    pagesize=(page_width, page_height),
    leftMargin=left_margin,
    rightMargin=right_margin,
    topMargin=top_margin,
    bottomMargin=bottom_margin
)

# Create a list to hold all elements
elements = []
main_title = Paragraph("Top 50 deliveries by value/volume non-FnO FnO and SME", getSampleStyleSheet()['Title'])
elements.extend([main_title])

# Iterate through your dataframes and add tables to the content frame
for i, df in enumerate(dfs):
    df.name = report_titles[i]
    df_name = df.name
    if i == 2 or i == 3:
        dfs[i] = df[fno_columns_to_select]
    else:
        dfs[i] = df[columns_to_select]

    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 10
    title = Paragraph(report_titles[i], style=h2_style)
    table_width = doc.width - doc.leftMargin - doc.rightMargin
    table = create_table_with_header(dfs[i], add_column_names=True, table_width=table_width, df_name=df_name)

    # Add elements for title and table
    records_per_page = 30
    chunks = [dfs[i][j:j + records_per_page] for j in range(0, len(dfs[i]), records_per_page)]
    for chunk in chunks:
        elements.extend([title, create_table_with_header(chunk, add_column_names=True, table_width=table_width, df_name=df_name), PageBreak()])

# Assign the function to be called for each page
doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)


In [35]:
##columns selection for printing
columns_to_select = ['Company Name', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value', 'NSE_Code']
col_length = [40,15,12,18,10]
fno_columns_to_select = ['NSE_Code', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value']



print("Top 50 deliveries by value/volume non-FnO FnO and SME ", nse_dely_date)
print("")
print("")
#print(nse_dely_date)
#print()
print("                          Top 50 delivery by value NON FNO ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")
print_formatted_rows(df_value_50,columns_to_select)
print("")
print("")
print("")


print("                        Top 50 delivery by volume NON FNO ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")

print_formatted_rows(df_volume_50,columns_to_select)
print("")
print("")
print("")


#### printing for fno
print("          Top 50 FNO Delivery by value ", nse_dely_date)
print("")
print("NSECode       ",'  Dely_Qty','      Price','   PrevClose','    Value' )
print("")
fno_formatted_rows(fno_value_50,fno_columns_to_select)
print("")
print("")
print("")

print("             Top 50 FNO Delivery by volume ", nse_dely_date)
print("")
print("NSECode       ",'  Dely_Qty','      Price','   PrevClose','   Value' )
print("")

fno_formatted_rows(fno_volume_50,fno_columns_to_select)
print("")
print("")
print("")


print("                          Top 50 delivery by value SME ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")
print_formatted_rows(sme_value_50,columns_to_select)
print("")
print("")
print("")




Top 50 deliveries by value/volume non-FnO FnO and SME  231011


                          Top 50 delivery by value NON FNO  231011

Company Name                                   Dely_Qty       Price    PrevClose            Value    NSECode

Zomato Ltd.                                  43,989,533      109.00      106.05         47,948.59    ZOMATO    
JIO Financial Services                        9,416,889      225.10      220.15         21,197.42    JIOFIN    
One97 Communications Ltd.                     2,095,051      971.75      949.70         20,358.66    PAYTM     
Suzlon Energy Ltd.                           70,919,227       27.65       26.35         19,609.17    SUZLON    
                                              1,837,430    1,000.00      999.99         18,374.30    LIQUIDBEES

Varun Beverages Ltd.                          1,726,434      917.65      927.95         15,842.62    VBL       
Gujarat Mineral Development Co                3,902,744      384.00      382.35      

In [36]:
sorted_df = bse_dely_data.sort_values(by='BSE_Delivery', ascending = False)
sorted_df


,BSE_Code,Date,BSE_Delivery
2325,532822,231011,36068464
2230,532667,231011,25553355
2205,532627,231011,15281994
2294,532775,231011,14032494
1715,530961,231011,11728850
...,...,...,...
3716,543853,231011,0
3934,936386,231011,0
3717,543858,231011,0
3965,937101,231011,0


In [37]:
m_segment = final_master_file[final_master_file['Segment'] == "SM"]
m_segment

,Date,Company Name,NSE_Code,BSE_Code,Total_Volume,Total_Delivery,Price,PrevClose,Total_Traded_Value,NSE_Volume,NSE_Delivery,BSE_Volume,BSE_Delivery,Segment
70,231011,Varanium Cloud Ltd.,CLOUD,,2540000,1113000,229.85,191.55,2558.2305,2540000.0,1113000.0,0.0,0.0,SM


In [126]:
sme_value_50

,Date,Company Name,NSE_Code,BSE_Code,Total_Volume,Total_Delivery,Price,PrevClose,Total_Traded_Value,NSE_Volume,NSE_Delivery,BSE_Volume,BSE_Delivery,Segment
33,230222,Globesecure Technologies Ltd.,GSTL,,562000,190000,142.45,140.30,270.655000,562000.0,190000.0,0.0,0.0,SM
125,230222,Inflame Appliances Ltd.,541083,541083,61250,51250,446.75,439.70,228.959375,0.0,0.0,61250.0,51250.0,M
14,230222,Varanium Cloud Ltd.,CLOUD,,18000,13000,1223.35,1285.80,159.035500,18000.0,13000.0,0.0,0.0,SM
77,230222,Shera Energy Ltd.,SHERA,,242000,216000,62.45,65.30,134.892000,242000.0,216000.0,0.0,0.0,ST
21,230222,Ducol Organics & Colours Ltd.,DUCOL,,131200,116800,114.20,106.40,133.385600,131200.0,116800.0,0.0,0.0,SM
199,230222,Droneacharya Aerial Innovation,543713,543713,114000,84000,156.60,164.80,131.544000,0.0,0.0,114000.0,84000.0,M
207,230222,Gensol Engineering Ltd.,GENSOL,542851,17860,13072,903.15,950.65,118.059768,0.0,0.0,17860.0,13072.0,M
65,230222,Perfect Infraengineers Ltd.,PERFECT,,540000,534000,19.85,18.95,105.999000,540000.0,534000.0,0.0,0.0,SM
206,230222,Indong Tea Company Ltd.,543769,543769,484000,484000,21.80,21.80,105.512000,0.0,0.0,484000.0,484000.0,MT
205,230222,Earthstahl & Alloys Ltd.,543765,543765,255000,183000,57.55,61.00,105.316500,0.0,0.0,255000.0,183000.0,M
